In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

from data_util import *

from data_generator import DataGenerator

import model as model_util

from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt

from numpy import array
import numpy as np



In [2]:
def reset_seed():
    seed_value= 0
    # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
    import os
    os.environ['PYTHONHASHSEED']=str(seed_value)
    # 2. Set `python` built-in pseudo-random generator at a fixed value
    import random
    random.seed(seed_value)
    # 3. Set `numpy` pseudo-random generator at a fixed value
    import numpy as np
    np.random.seed(seed_value)
    # 4. Set the `tensorflow` pseudo-random generator at a fixed value
    import tensorflow as tf
    tf.random.set_seed(seed_value)

reset_seed()

path = "./data/"
model_path = path + "stock.h5"

In [3]:
trainX, trainY, positiveX, positiveY, negativeX, negativeY = load_data("", "train", path, balanced = True)
valX, valY, positiveX_, positiveY_, negativeX_, negativeY_ = load_data("", "Val", path, balanced = False)

In [4]:
dim = 10
seqs = 30
dropout = 0.1
lays = 5
lays_seq = 1

In [14]:
# https://machinelearningmastery.com/how-to-use-the-timeseriesgenerator-for-time-series-forecasting-in-keras/
# define generator
batch_size = 512

train_generator = get_gen(trainX, trainY, shuffle=False, time_steps = seqs)

val_generator = get_gen(valX, valY, shuffle=False, time_steps = seqs)

x_train, y_train = create_dataset(trainX, trainY, seqs)

x_val, y_val = create_dataset(valX, valY, seqs)


# number of samples
print('Samples: %d' % len(train_generator))


print(x_train.shape)
print(y_train.shape)

features = x_train.shape[-1]

print("Feature: {}".format(features))


Samples: 6576
(420854, 30, 35)
(420854, 2)
Feature: 35


In [15]:
x_train[90].shape

(30, 35)

In [16]:
#model.summary()
import tensorflow as tf


use_regular = True #@param {type:"boolean"}

checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             load_weights_on_restart=True)

callbacks_list = []

epochs = 100

if (use_regular):
    print("Training on: {} val {}".format(len(y_train), len(y_val)))
    history = model.fit(
        x=x_train,
        y=y_train,
        validation_data=(x_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1,
        callbacks=callbacks_list
    )
else:
    print("Training on: {} val {}".format(len(train_generator)*batch_size, len(val_generator)))
    history = model.fit(
        x=train_generator,
        validation_data = val_generator,
        steps_per_epoch = len(train_generator),
        epochs=epochs,
        verbose=1,
        callbacks=callbacks_list
    )


Training on: 420854 val 14934
Epoch 1/100
822/822 [==============================] - 67s 75ms/step - loss: 0.7273 - accuracy: 0.5000 - val_loss: 0.7965 - val_accuracy: 0.1170
Epoch 2/100
822/822 [==============================] - 60s 73ms/step - loss: 0.7236 - accuracy: 0.5009 - val_loss: 0.7851 - val_accuracy: 0.1323
Epoch 3/100
822/822 [==============================] - 60s 72ms/step - loss: 0.7197 - accuracy: 0.5010 - val_loss: 0.7744 - val_accuracy: 0.1507
Epoch 4/100
822/822 [==============================] - 59s 72ms/step - loss: 0.7165 - accuracy: 0.5007 - val_loss: 0.7642 - val_accuracy: 0.1684
Epoch 5/100
822/822 [==============================] - 60s 72ms/step - loss: 0.7139 - accuracy: 0.5001 - val_loss: 0.7554 - val_accuracy: 0.1866
Epoch 6/100
168/822 [=====>........................] - ETA: 45s - loss: 0.7130 - accuracy: 0.4994

KeyboardInterrupt: 

In [ ]:
def show_loss(history): 
    plt.plot(history.history['loss'])
    if (history.history['val_loss']):
        plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

show_loss(history)

In [ ]:
def show_accuracy(history):
    plt.plot(history.history['accuracy'])
    if (history.history['val_accuracy']):
        plt.plot(history.history['val_accuracy'])
    plt.title('accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

show_accuracy(history)

In [ ]:


#model_loaded = load_model(filepath = filepath_encoder)
model_loaded = model

to_int = np.vectorize(lambda x: int(round(x)))

In [ ]:

posX_, posY_ = create_dataset(positiveX_, positiveY_, seqs)
negX_, negY_ = create_dataset(negativeX_, negativeY_, seqs)



In [ ]:
print(model_loaded.predict(negX_))

In [ ]:
pos_final = to_int(model_loaded.predict(posX_))
neg_final = to_int(model_loaded.predict(negX_))


pos_final

In [ ]:
model_loaded.evaluate(posX_, posY_)

In [ ]:
def check(list):
    pos_count = 0
    neg_count = 0
    for x in list:
        if (x == 1):
            pos_count += 1
        else:
            neg_count += 1
    print("Pos: {} Neg {}".format(pos_count, neg_count))

In [ ]:
print("Pos")
check(pos_final)
print("Neg")
check(neg_final)


In [ ]:
model_loaded.predict(posX_)[:10]

In [ ]:
model_loaded.predict(negX_)[:10]

In [ ]:
model_loaded = model
valX, valY, positiveX_, positiveY_, negativeX_, negativeY_ = load_data("btcusd", "Val", path, False)
model_loaded.evaluate(get_gen(valX, valY, shuffle=False, time_steps = seqs)) 

valX, valY, positiveX_, positiveY_, negativeX_, negativeY_ = load_data("ethusd", "Val", path, False)
model_loaded.evaluate(get_gen(valX, valY, shuffle=False, time_steps = seqs)) 

valX, valY, positiveX_, positiveY_, negativeX_, negativeY_ = load_data("btcusd17", "Val", path, False)
model_loaded.evaluate(get_gen(valX, valY, shuffle=False, time_steps = seqs)) 

valX, valY, positiveX_, positiveY_, negativeX_, negativeY_ = load_data("btcusdAug19", "Val", path, False)
model_loaded.evaluate(get_gen(valX, valY, shuffle=False, time_steps = seqs)) 


In [ ]:
X = list()
Y = list()
X = [x+1 for x in range(200)]
Y = [y for y in X]

In [ ]:
generator = get_gen(X, Y, shuffle=True, batch_size=20)

x, y = create_dataset(X, Y, seqs)

index = 100

In [12]:
#Model 

from tensorflow.keras.layers import BatchNormalization, GlobalMaxPool1D, Bidirectional, Dense, Flatten, Conv2D, LeakyReLU, Dropout, LSTM, GRU, Input
from tensorflow.keras import Model, Sequential
from tensorflow.keras import datasets, layers, models

import tensorflow as tf
#import tensorflow_addons as tfa
 

#dim = 600
att = 15

def add_attention(x, d_model = 512):
    query = tf.keras.layers.Dense(d_model)(x)  
    value = tf.keras.layers.Dense(d_model)(x)
    key = tf.keras.layers.Dense(d_model)(x)
    attention = tf.keras.layers.Attention()([query, value, key]) 
    x = tf.keras.layers.Add()([x, attention])
    x = tf.keras.layers.LayerNormalization()(x)
    return x

def add_deep_layers(input_layer, units = dim):
    x = Dense(units)(input_layer)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(units, activation='tanh')(x)
    x = Dropout(0.2)(x)
    x = Dense(units, activation='swish')(x)
    return x

def get_model(seqs, features):
    reset_seed()
    inputX = Input(shape=(
        seqs,
        features
        )
    )
    x = inputX
    x = add_attention(x, features)
    x = LSTM(units=att, return_sequences=True)(x)
    x = add_deep_layers(x, dim)
    x = LSTM(dim, return_sequences=False)(x)
    x = add_deep_layers(x, dim)
    x = add_deep_layers(x, dim)
    x = add_deep_layers(x, dim)
    x = add_deep_layers(x, dim)
    x = add_deep_layers(x, 20)
    x = Dense(10)(x)
    x = Dense(2, activation='sigmoid')(x)

    model = Model(inputs=[inputX], outputs=x)
    
    return model

    
def load_model(filepath = "stock.h5"):
    model_loaded = tf.keras.models.load_model(filepath)
    return model_loaded

In [13]:


#model = load_model(filepath = "drive/My Drive/model/stock.h5")

#!rm "drive/My Drive/model/encoder.h5"

model = get_model(seqs, features)

#radam = tfa.optimizers.RectifiedAdam()
#ranger = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)

model.compile(
    #optimizer=ranger,
    optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.001, rho=0.95, epsilon=1e-07),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 30, 35)]     0                                            
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 30, 35)       1260        input_3[0][0]                    
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 30, 35)       1260        input_3[0][0]                    
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 30, 35)       1260        input_3[0][0]                    
______________________________________________________________________________________________

In [ ]:
!rm "stock.h5"